In [3]:
import os
from PIL import Image

import numpy as np

np.random.seed(seed=1)

from torchvision import transforms

In [4]:
class CamVidDataSet_open:
    def __init__(self, imgs_path, labels_path, transform):
        self.imgs_path = imgs_path
        self.labels_path = labels_path
        self.transform = transform

        self.imgs = os.listdir(self.imgs_path)
        self.labels = list(
            map(lambda x: x[:-4] + "_L.png", self.imgs)
        )  # see train_labels

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.imgs_path, self.imgs[idx])
        image = Image.open(img_loc).convert("RGB")
        image_out = self.transform(image)

        label_loc = os.path.join(self.labels_path, self.labels[idx])
        label = Image.open(label_loc).convert("RGB")
        label_out = self.transform(label)

        return image_out, label_out, self.imgs[idx], self.labels[idx]

In [5]:
input_size = (128, 128)
transformation = transforms.Compose([transforms.Resize(input_size, 0)])
camvid = CamVidDataSet_open(
    "./CamVid/train/", "./CamVid/train_labels/", transform=transformation
)

In [25]:
# Defining endings and augmentation

# _00: original data
# _01: horizontal transformation
# _02: shape transformation (resize and crop)
# _03: brightness transformation
# _04: contrast transformation
# _05: gaussian noise
# _06: total: all transformations

In [6]:
# Defining all transformations

aug_horizontal = transforms.RandomHorizontalFlip(p=1)

shape_aug = transforms.RandomResizedCrop((128, 128), scale=(0.1, 0.9), ratio=(0.5, 2))

brightness_aug = transforms.ColorJitter(brightness=0.5, contrast=0, saturation=0, hue=0)

contrast_aug = transforms.ColorJitter(
    brightness=0, contrast=0.5, saturation=0.2, hue=0.1
)


class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=0.001):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + "(mean={0}, std={1})".format(
            self.mean, self.std
        )


transform_gaussian = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(0, 1),
        AddGaussianNoise(
            0.0, 0.001
        ),  # Change 0.001 to be a higher number of we need more noise - shall also be done in the AddGaussia
        transforms.ToPILImage(),
    ]
)

total_aug_data = transforms.Compose(
    [aug_horizontal, shape_aug, brightness_aug, contrast_aug, transform_gaussian]
)

total_aug_labels = transforms.Compose([aug_horizontal, shape_aug])

In [9]:
path_augment = "./CamVid/train_augment/"
path_labels_augmemt = "./CamVid/train_labels_augment/"

In [10]:
for i in range(len(camvid)):
    # Define current image and labels
    img = camvid[i][0]
    labels = camvid[i][1]
    name = camvid[i][2]
    name_labels = camvid[i][3]

    # Save original data and labels
    img.save(path_augment + name + "_00", format="png")
    labels.save(path_labels_augmemt + name_labels + "_00", format="png")

    # Horizontal flip
    img_flip = aug_horizontal(img)
    labels_flip = aug_horizontal(labels)

    img_flip.save(path_augment + name + "_01", format="png")
    labels_flip.save(path_labels_augmemt + name_labels + "_01", format="png")

    # Crop and resize
    rand_1 = np.random.randint(1000)

    torch.manual_seed(rand_1)
    img_crop = shape_aug(img)

    torch.manual_seed(rand_1)
    labels_crop = shape_aug(labels)

    img_crop.save(path_augment + name + "_02", format="png")
    labels_crop.save(path_labels_augmemt + name_labels + "_02", format="png")

    # Brightness
    img_bright = brightness_aug(img)

    img_bright.save(path_augment + name + "_03", format="png")
    labels.save(path_labels_augmemt + name_labels + "_03", format="png")

    # Colour and saturation
    img_contrast = contrast_aug(img)

    img_contrast.save(path_augment + name + "_04", format="png")
    labels.save(path_labels_augmemt + name_labels + "_04", format="png")

    # Gaussian noise

    img_gaussian = transform_gaussian(img)

    img_gaussian.save(path_augment + name + "_05", format="png")
    labels.save(path_labels_augmemt + name_labels + "_05", format="png")

    # Combined augmentation
    rand_2 = np.random.randint(1000)

    torch.manual_seed(rand_2)
    img_combined = total_aug_data(img)

    torch.manual_seed(rand_2)
    labels_combined = total_aug_labels(labels)

    img_combined.save(path_augment + name + "_06", format="png")
    labels_combined.save(path_labels_augmemt + name_labels + "_06", format="png")